In [20]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [2]:
df = pd.read_csv("../data/ufc-master.csv")

In [3]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5540 entries, 0 to 5539
Data columns (total 79 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   R_fighter                     4252 non-null   object 
 1   B_fighter                     4252 non-null   object 
 2   R_odds                        4252 non-null   float64
 3   B_odds                        4252 non-null   float64
 4   R_ev                          4252 non-null   float64
 5   B_ev                          4252 non-null   float64
 6   date                          4252 non-null   object 
 7   location                      4252 non-null   object 
 8   country                       4252 non-null   object 
 9   Winner                        4252 non-null   object 
 10  title_bout                    4252 non-null   object 
 11  weight_class                  4252 non-null   object 
 12  gender                        4252 non-null   object 
 13  no_

In [4]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [5]:
display(df[['Winner', 'label']])
df.info(verbose=True)

,Winner,label
0,Red,0
1,Red,0
2,Red,0
3,Red,0
4,Red,0
...,...,...
4247,Blue,1
4248,Red,0
4249,Red,0
4250,Red,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4179 entries, 0 to 4251
Data columns (total 80 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   R_fighter                     4179 non-null   object 
 1   B_fighter                     4179 non-null   object 
 2   R_odds                        4179 non-null   float64
 3   B_odds                        4179 non-null   float64
 4   R_ev                          4179 non-null   float64
 5   B_ev                          4179 non-null   float64
 6   date                          4179 non-null   object 
 7   location                      4179 non-null   object 
 8   country                       4179 non-null   object 
 9   Winner                        4179 non-null   object 
 10  title_bout                    4179 non-null   object 
 11  weight_class                  4179 non-null   object 
 12  gender                        4179 non-null   object 
 13  no_

In [6]:
#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

In [7]:
#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

In [8]:
#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models
df_train = df[:-200]
odds_train = odds_df[:-200]
label_train = label_df[:-200]

df_test = df[-200:]
odds_test = odds_df[-200:]
label_test = label_df[-200:]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

200
200
200
3979
3979
3979


In [9]:
print(df_train.columns)

Index(['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev', 'date',
       'location', 'country', 'Winner', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
  

In [10]:
#Full list of possible features
#Basically inclusive features with the labelm constant_1 and Winner removed
my_pos_features = ['R_fighter', 'B_fighter', 'R_odds', 'B_odds', 'R_ev', 'B_ev',
       'location', 'country', 'title_bout', 'weight_class', 'gender',
       'no_of_rounds', 'B_current_lose_streak', 'B_current_win_streak',
       'B_draw', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT',
       'B_avg_TD_landed', 'B_avg_TD_pct', 'B_longest_win_streak', 'B_losses',
       'B_total_rounds_fought', 'B_total_title_bouts',
       'B_win_by_Decision_Majority', 'B_win_by_Decision_Split',
       'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission',
       'B_win_by_TKO_Doctor_Stoppage', 'B_wins', 'B_Stance', 'B_Height_cms',
       'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak',
       'R_current_win_streak', 'R_draw', 'R_avg_SIG_STR_landed',
       'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct',
       'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought',
       'R_total_title_bouts', 'R_win_by_Decision_Majority',
       'R_win_by_Decision_Split', 'R_win_by_Decision_Unanimous',
       'R_win_by_KO/TKO', 'R_win_by_Submission',
       'R_win_by_TKO_Doctor_Stoppage', 'R_wins', 'R_Stance', 'R_Height_cms',
       'R_Reach_cms', 'R_Weight_lbs', 'R_age', 'B_age', 'lose_streak_dif',
       'win_streak_dif', 'longest_win_streak_dif', 'win_dif', 'loss_dif',
       'total_round_dif', 'total_title_bout_dif', 'ko_dif', 'sub_dif',
       'height_dif', 'reach_dif', 'age_dif', 'sig_str_dif', 'avg_sub_att_dif',
       'avg_td_dif', 'empty_arena']

In [11]:
test_model = DecisionTreeClassifier(random_state=75)
test_current_features = []

In [12]:
#Input: American Odds, and Probability of a Winning Bet
#Output: Bet EV based on a $100 bet
def get_bet_ev(odds, prob):
    if odds>0:
        return ((odds * prob) - (100 * (1-prob)) )
    else:
        return ((100 / abs(odds))*100*prob - (100 * (1-prob)))


In [13]:
#Input: American Odds
#Output: Profit on a successful bet
def get_bet_return(odds):
    if odds>0:
        return odds
    else:
        return (100 / abs(odds))*100


In [14]:
def get_ev_from_df(ev_df, print_stats = False, min_ev = 0):
    num_matches = 0
    num_bets = 0
    num_wins = 0
    num_losses= 0
    num_under= 0
    num_under_losses = 0
    num_under_wins = 0
    num_even = 0
    num_even_losses = 0
    num_even_wins = 0
    num_fav = 0
    num_fav_wins = 0
    num_fav_losses = 0
    profit = 0
    profit_per_bet = 0
    profit_per_match = 0    

    for index, row in ev_df.iterrows():
        num_matches = num_matches+1
        t1_bet_ev = get_bet_ev(row['t1_odds'], row['t1_prob'])
        #print(f"ODDS:{row['t1_odds']} PROB: {row['t1_prob']} EV: {t1_bet_ev}")
        t2_bet_ev = get_bet_ev(row['t2_odds'], row['t2_prob'])
        #print(f"ODDS:{row['t2_odds']} PROB: {row['t2_prob']} EV: {t2_bet_ev}")
        #print()
        
        t1_bet_return = get_bet_return(row['t1_odds'])
        t2_bet_return = get_bet_return(row['t2_odds'])
        
        
        if (t1_bet_ev > min_ev or t2_bet_ev > min_ev):
            num_bets = num_bets+1

            
        if t1_bet_ev > min_ev:
            if row['winner'] == 0:
                num_wins += 1
                profit = profit + t1_bet_return
                #print(t1_bet_return)
            elif row['winner'] == 1:
                num_losses += 1
                profit = profit - 100
            if (t1_bet_return > t2_bet_return):
                num_under += 1
                if row['winner'] == 0:
                    num_under_wins += 1
                elif row['winner'] == 1:
                    num_under_losses += 1
            elif (t1_bet_return < t2_bet_return):
                num_fav += 1
                if row['winner'] == 0:
                    num_fav_wins += 1
                elif row['winner'] == 1:
                    num_fav_losses += 1
            else:
                num_even += 1
                if row['winner'] == 0:
                    num_even_wins += 1
                elif row['winner'] == 1:
                    num_even_losses += 1

        if t2_bet_ev > min_ev:
            if row['winner'] == 1:
                num_wins += 1                    
                profit = profit + t2_bet_return
            elif row['winner'] == 0:
                num_losses += 1
                profit = profit - 100
            if (t2_bet_return > t1_bet_return):
                num_under += 1
                if row['winner'] == 1:
                    num_under_wins += 1
                elif row['winner'] == 0:
                    num_under_losses += 1
            elif (t2_bet_return < t1_bet_return):
                num_fav += 1
                if row['winner'] == 1:
                    num_fav_wins += 1
                elif row['winner'] == 0:
                    num_fav_losses += 1
            else:
                num_even += 1
                if row['winner'] == 1:
                    num_even_wins += 1
                elif row['winner'] == 0:
                    num_even_losses += 1
            
    profit_per_bet = profit / num_bets
    profit_per_match = profit / num_matches
        
        
    if print_stats:
        print(f"""
          Number of matches: {num_matches}
          Number of bets: {num_bets}
          Number of winning bets: {num_wins}
          Number of losing bets: {num_losses}
          Number of underdog bets: {num_under}
          Number of underdog wins: {num_under_wins}
          Number of underdog losses: {num_under_losses}
          Number of Favorite bets: {num_fav}
          Number of favorite wins: {num_fav_wins}
          Number of favorite losses: {num_fav_losses}
          Number of even bets: {num_even}
          Number of even wins: {num_even_wins}
          Number of even losses: {num_even_losses}
          Profit: {profit}
          Profit per bet: {profit_per_bet}
          Profit per match: {profit_per_match}
          
          """)
        
    return (profit_per_bet)

In [15]:
#INPUT: 
#df: The df to be evaluated
#m: The model to use
# labels: The labels
#odds: The odds
#min_ev: The minimum EV to place a bet
def custom_cv_eval(df, m, labels, odds, min_ev=0):
    X = np.array(df)
    y = np.array(labels)
    odds = np.array(odds)
    running_total = 0
    count=1
    kf = KFold(n_splits=5, shuffle=True, random_state=75)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        odds_train, odds_test = odds[train_index], odds[test_index]
        #display(y_train)
        m.fit(X_train, y_train)
        probs=m.predict_proba(X_test)
        #print(probs)
        #We need to prep the dataframe to evaluate....
        #X_odds = X_test[['t1_odds', 't2_odds']]
        #print(X_test)
        #print(X_test[:, -1])
        #print(X_test[:, -2])
        X_odds = list(zip(odds_test[:, -2], odds_test[:, -1], probs[:, 0], probs[:, 1], y_test))
        ev_prepped_df = pd.DataFrame(X_odds, columns=['t1_odds', 't2_odds', 't1_prob', 't2_prob', 'winner'])
        #display(ev_prepped_df)
        #display(temp_df)
        #print(f"{count}: {get_ev_from_df(ev_prepped_df, print_stats = False)}")
        count=count+1
        running_total = running_total + get_ev_from_df(ev_prepped_df, print_stats = False, min_ev = min_ev)
        #display(ev_prepped_df)
    
    return running_total

In [16]:
custom_cv_eval(df_train[['constant_1']], test_model, label_train, odds_train)

-23.547452012267335

In [17]:
print(df_train[['label', 'Winner']])

      label Winner
0         0    Red
1         0    Red
2         0    Red
3         0    Red
4         0    Red
...     ...    ...
4045      1   Blue
4046      0    Red
4048      1   Blue
4049      1   Blue
4050      1   Blue

[3979 rows x 2 columns]


In [18]:
#We split off the labels and the odds.  Now we can rewrite the get_best_features function
#INPUT
#pos_features: The list of possible features
#m: The model
#cur_features: The list of current features
#scale: Does the data need to be scaled?  


def get_best_features(pos_features, m, df, cur_features, labels, odds, scale=False):
    best_feature = ''
        
    #If there are no current features...
    if len(cur_features) == 0:
        best_score = -100
    else:
        df_sel = df[cur_features]
        df_sel = df_sel.dropna()
        df_sel = pd.get_dummies(df_sel)
        #OK we need to filter the labels and odds based off of the indices
        labels_sel = labels[labels.index.isin(df_sel.index)]
        odds_sel = odds[odds.index.isin(df_sel.index)]        
        best_score = custom_cv_eval(df_sel, m, labels_sel, odds_sel)
        
    best_feature = ""
    
    print(f"Current best score is: {best_score}")
    #Go thru every feature and test it...
    for f in pos_features:
        #If f is not a current feature
        if f not in cur_features:
            new_features = [f] + cur_features
            df_sel = df[new_features]
            df_sel = df_sel.dropna()
            df_sel = pd.get_dummies(df_sel)
            #display(df_sel)
            #OK we need to filter the labels and odds based off of the indices
            labels_sel = labels[labels.index.isin(df_sel.index)]
            odds_sel = odds[odds.index.isin(df_sel.index)]
            new_score = custom_cv_eval(df_sel, m, labels_sel, odds_sel)
            #print(f"{len(df_sel)} {len(labels_sel)} {len(odds_sel)}")
            print(f"Feature: {f} Score: {new_score}")
            if new_score > best_score:
                best_score = new_score
                best_feature = f
    if best_feature != "":
        print(f"The best feature was {best_feature}.  It scored {best_score}")
        cur_features = [best_feature] + cur_features
        #Keep running until we don't improve
        return(get_best_features(pos_features, m, df, cur_features, labels, odds, scale))
    else:
        print("NO IMPROVEMENT")
        print(f"FINAL BEST SCORE: {best_score}")
        return cur_features                
                
    return []



In [19]:
(get_best_features(my_pos_features, test_model, df_train, [], label_train, odds_train, True))

Current best score is: -100
Feature: R_fighter Score: -12.359255573051662
Feature: B_fighter Score: -9.506234185490857
Feature: R_odds Score: -30.48964440616377
Feature: B_odds Score: -23.735402291758316
Feature: R_ev Score: -30.48964440616377
Feature: B_ev Score: -23.735402291758316
Feature: location Score: -18.419847335276092
Feature: country Score: -18.403005110823894
Feature: title_bout Score: -27.24151065686665
Feature: weight_class Score: -27.391509271667292
Feature: gender Score: -26.054925736138625
Feature: no_of_rounds Score: -29.51557254815579
Feature: B_current_lose_streak Score: -24.897844534155606
Feature: B_current_win_streak Score: -25.247830375022673
Feature: B_draw Score: -23.052064918509913
Feature: B_avg_SIG_STR_landed Score: -11.986200607911503
Feature: B_avg_SIG_STR_pct Score: -28.010063387342992
Feature: B_avg_SUB_ATT Score: -24.75124763812522
Feature: B_avg_TD_landed Score: -15.95157696635745
Feature: B_avg_TD_pct Score: -1.2167213608852645
Feature: B_longest_win

KeyboardInterrupt: 

In [21]:
model_1 = LogisticRegression(random_state=75)

In [22]:
(get_best_features(my_pos_features, model_1, df_train, [], label_train, odds_train, True))

Current best score is: -100
Feature: R_fighter Score: -27.73529016456652
Feature: B_fighter Score: -30.874346971243533
Feature: R_odds Score: -28.532909112616146
Feature: B_odds Score: -25.623022194215533
Feature: R_ev Score: -32.45690584722519
Feature: B_ev Score: 9.989235671762625
Feature: location Score: -19.54117263713706
Feature: country Score: -18.440820558392154
Feature: title_bout Score: -26.66039359930254
Feature: weight_class Score: -27.252708486822005
Feature: gender Score: -26.176509738245272
Feature: no_of_rounds Score: -29.955700861461093
Feature: B_current_lose_streak Score: -25.118183762666174
Feature: B_current_win_streak Score: -18.07997775287799
Feature: B_draw Score: -23.66931154824347
Feature: B_avg_SIG_STR_landed Score: -15.174827490268731
Feature: B_avg_SIG_STR_pct Score: -15.860076955399895
Feature: B_avg_SUB_ATT Score: -16.876934456161614
Feature: B_avg_TD_landed Score: -17.854398687481208
Feature: B_avg_TD_pct Score: -15.550316005039228
Feature: B_longest_win_

D:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_

Feature: location Score: 10.798810110499991


D:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
D:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_

Feature: country Score: -2.450333154524963
Feature: title_bout Score: 1.7810558333393027


...

D:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Feature: sub_dif Score: 19.484529542909705
Feature: height_dif Score: 22.761250514799112
Feature: reach_dif Score: 19.95026383488789
Feature: age_dif Score: 22.64830699691665


D:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Feature: sig_str_dif Score: 15.924475681873556
Feature: avg_sub_att_dif Score: 14.15128894405349
Feature: avg_td_dif Score: 20.587795179790852
Feature: empty_arena Score: 30.50034217280308
NO IMPROVEMENT
FINAL BEST SCORE: 31.158123326360922


['B_draw', 'B_longest_win_streak', 'total_round_dif', 'R_ev', 'B_ev']

In [23]:
model_2 = DecisionTreeClassifier(random_state=75)
(get_best_features(my_pos_features, model_2, df_train, [], label_train, odds_train, True))

Current best score is: -100
Feature: R_fighter Score: -12.359255573051662
Feature: B_fighter Score: -9.506234185490857
Feature: R_odds Score: -30.48964440616377
Feature: B_odds Score: -23.735402291758316
Feature: R_ev Score: -30.48964440616377
Feature: B_ev Score: -23.735402291758316
Feature: location Score: -18.419847335276092
Feature: country Score: -18.403005110823894
Feature: title_bout Score: -27.24151065686665
Feature: weight_class Score: -27.391509271667292
Feature: gender Score: -26.054925736138625
Feature: no_of_rounds Score: -29.51557254815579
Feature: B_current_lose_streak Score: -24.897844534155606
Feature: B_current_win_streak Score: -25.247830375022673
Feature: B_draw Score: -23.052064918509913
Feature: B_avg_SIG_STR_landed Score: -11.986200607911503
Feature: B_avg_SIG_STR_pct Score: -28.010063387342992
Feature: B_avg_SUB_ATT Score: -24.75124763812522
Feature: B_avg_TD_landed Score: -15.95157696635745
Feature: B_avg_TD_pct Score: -1.2167213608852645
Feature: B_longest_win

Feature: gender Score: 23.280604157217322
Feature: no_of_rounds Score: 15.23408626547258
Feature: B_current_lose_streak Score: 18.314949003290888
Feature: B_current_win_streak Score: 7.93300115684214
Feature: B_draw Score: 20.32825253093563
Feature: B_avg_SIG_STR_landed Score: -8.290847329512983
Feature: B_avg_SIG_STR_pct Score: 18.9337443500226
Feature: B_avg_SUB_ATT Score: 13.167746047557573
Feature: B_avg_TD_landed Score: 18.20091541265
Feature: B_longest_win_streak Score: 11.79385216258066
Feature: B_losses Score: 19.555248647679832
Feature: B_total_rounds_fought Score: 15.970477194164896
Feature: B_total_title_bouts Score: 17.492783488463232
Feature: B_win_by_Decision_Majority Score: 21.398476591527647
Feature: B_win_by_Decision_Split Score: 14.866083559482547
Feature: B_win_by_Decision_Unanimous Score: 4.512437262415703
Feature: B_win_by_KO/TKO Score: 12.218866274287969
Feature: B_win_by_Submission Score: 15.177766487360357
Feature: B_win_by_TKO_Doctor_Stoppage Score: 20.04896768

['R_Stance', 'B_fighter', 'B_avg_TD_pct']

In [24]:
model_3 = RandomForestClassifier(random_state=75)
(get_best_features(my_pos_features, model_3, df_train, [], label_train, odds_train, True))

Current best score is: -100
Feature: R_fighter Score: -31.764793166121848
Feature: B_fighter Score: 0.7197311658623571
Feature: R_odds Score: -28.379408874251624
Feature: B_odds Score: -24.034549233468184
Feature: R_ev Score: -28.379408874251624
Feature: B_ev Score: -23.883836699188805
Feature: location Score: -18.59228748140632
Feature: country Score: -18.139037231606686
Feature: title_bout Score: -25.589058200226646
Feature: weight_class Score: -26.481946432780873
Feature: gender Score: -26.25545210080947
Feature: no_of_rounds Score: -30.08455109573676
Feature: B_current_lose_streak Score: -24.5753198013967
Feature: B_current_win_streak Score: -25.6196856762303
Feature: B_draw Score: -22.725890421865618
Feature: B_avg_SIG_STR_landed Score: -11.995412998298047
Feature: B_avg_SIG_STR_pct Score: -28.628300191341545
Feature: B_avg_SUB_ATT Score: -25.973947359506457
Feature: B_avg_TD_landed Score: -16.088001782832155
Feature: B_avg_TD_pct Score: -4.250507519902718
Feature: B_longest_win_s

Feature: no_of_rounds Score: 10.195197537913245
Feature: B_current_lose_streak Score: 24.45089206109931
Feature: B_current_win_streak Score: 17.3780335189237
Feature: B_draw Score: 23.72307494818164
Feature: B_avg_SIG_STR_landed Score: 3.8720012537766557
Feature: B_avg_SUB_ATT Score: 9.949519031981472
Feature: B_avg_TD_landed Score: -7.206572333263947
Feature: B_avg_TD_pct Score: -1.9934488529270862
Feature: B_longest_win_streak Score: 12.76295218651001
Feature: B_losses Score: 12.46397221696613
Feature: B_total_rounds_fought Score: 12.707092837264462
Feature: B_total_title_bouts Score: 24.622936979632385
Feature: B_win_by_Decision_Majority Score: 17.04546810404878
Feature: B_win_by_Decision_Split Score: 16.538012234882075
Feature: B_win_by_Decision_Unanimous Score: 4.523109342507209
Feature: B_win_by_KO/TKO Score: 18.641815554958423
Feature: B_win_by_Submission Score: 17.28834422114783
Feature: B_win_by_TKO_Doctor_Stoppage Score: 17.95579183695302
Feature: B_wins Score: 9.353553335002

Feature: B_win_by_TKO_Doctor_Stoppage Score: 27.825328241622593
Feature: B_wins Score: 12.351431841578503
Feature: B_Stance Score: 24.744104914076054
Feature: B_Height_cms Score: 21.616841921332586
Feature: B_Reach_cms Score: 15.420341624718155
Feature: B_Weight_lbs Score: 23.907764516394664
Feature: R_current_lose_streak Score: 17.998093067780808
Feature: R_current_win_streak Score: 5.414904446967311
Feature: R_draw Score: 27.486754952740334
Feature: R_avg_SIG_STR_landed Score: 5.021469765803827
Feature: R_avg_SIG_STR_pct Score: 2.6128607988645554
Feature: R_avg_SUB_ATT Score: 16.44708266038666
Feature: R_avg_TD_landed Score: 7.890183683951859
Feature: R_avg_TD_pct Score: 9.31722635034292
Feature: R_longest_win_streak Score: 7.708918466495282
Feature: R_losses Score: 3.9144106620676906
Feature: R_total_rounds_fought Score: 3.3842485436478946
Feature: R_total_title_bouts Score: 21.717302637995704
Feature: R_win_by_Decision_Majority Score: 26.717331925127834
Feature: R_win_by_Decision_S

['B_current_lose_streak',
 'B_total_title_bouts',
 'B_avg_SIG_STR_pct',
 'B_fighter']